Installing PyMongo

In [9]:
!pip install pymongo tabulate


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


Connecting to MongoDB

In [3]:
from pymongo import MongoClient
from datetime import datetime

# Connect to the MongoDB client (replace with your connection URI if not local)
client = MongoClient("mongodb://localhost:27017/")

# Select the database and collection
db = client["WSDA_Music"]
invoice_collection = db["Invoice"]

## Challenge 1

General queries that begin to give you some high-level context.

### Q1. How many transactions took place between the years 2011 and 2012?

In [3]:
# Define the date range in the string format
start_date = '2011-01-01 00:00:00'
end_date = '2012-12-31 23:59:59'

# Query to count number of documents/records
number_of_transactions = invoice_collection.count_documents({
    "InvoiceDate": {
        "$gte": start_date,
        "$lte": end_date
    }
})

# Print number of transactions
print(f"{number_of_transactions} Transactions took place between 2011 and 2012")

167 Transactions took place between 2011 and 2012


### Q2. How much money did WSDA Music make during the same period?

In [4]:
# Define the date range in the string format
start_date = '2011-01-01 00:00:00'
end_date = '2012-12-31 23:59:59'

# Aggregation pipeline to filter and sum total sales
pipeline = [
    {
        "$match": {
            "InvoiceDate" : {
                "$gte": start_date,
                "$lte": end_date
            }
        }
    },
    {
        "$group": {
            "_id": None,
            "Total Sales": { "$sum" : "$Total"}
        }
    }
]
# Execute the aggregation pipeline on the 'invoice_collection' and convert the result to a list.
# The aggregation pipeline is stored in the 'pipeline' variable and is designed to filter documents by date and sum their sales.
total_sales_list = list(invoice_collection.aggregate(pipeline))

# Extract the total sales amount from the first element of the 'total_sales_list'.
total_sales = total_sales_list[0]['Total Sales']

print("Total Sales made between 2011 and 2012 is", total_sales)


Total Sales made between 2011 and 2012 is 1947.97


## Challenge 2

### Q1. How many transactions took place between the years 2011 and 2012?

In [ ]:
# Define the date range in the string format
start_date = '2011-01-01 00:00:00'
end_date = '2012-12-31 23:59:59'

# Aggregation pipeline
pipeline = [
    {
        "$lookup": {
            "from": "Customer",
            "localField": "CustomerId",
            "foreignField": "CustomerId",
            "as": "customerDetails"
        }
    },
    {
        "$unwind": "$customerDetails"
    },
    {
        "$match": {
            "InvoiceDate": {
                "$gte": start_date,
                "$lte": end_date
            }
        }
    },
    {
        "$group": {
            "_id": {
                "FirstName": "$customerDetails.FirstName",
                "LastName": "$customerDetails.LastName",
                "City": "$customerDetails.City",
                "State": "$customerDetails.State"
            },
            "Email": {"$first": "$customerDetails.Email"},
            "Address": {"$first": "$customerDetails.Address"}
        }
    },
    {
        "$sort": {"_id.FirstName": 1}
    }
]

# Execute the aggregation pipeline
customers = list(db.Invoice.aggregate(pipeline))

# Print the results
for customer in customers:
    full_name = f"{customer['_id']['FirstName']} {customer['_id']['LastName']}"
    address = customer['Address']
    city = customer['_id'].get('City', '')
    state = customer['_id'].get('State', '')
    email = customer['Email']

    print(f"Name: {full_name}")
    print(f"Address: {address}")
    print(f"City: {city}")
    print(f"State: {state}")
    print(f"Email: {email}")
    print()


#### Showing the results in tabular form

In [13]:
from tabulate import tabulate

# Execute the aggregation pipeline
customers = list(db.Invoice.aggregate(pipeline))

# Prepare data for tabulate
table_data = []
for customer in customers:
    full_name = f"{customer['_id']['FirstName']} {customer['_id']['LastName']}"
    address = customer['Address']
    city = customer['_id'].get('City', '')
    state = customer['_id'].get('State', '')
    email = customer['Email']

    table_data.append([full_name, address, city, state, email])

# Print results in tabular format
headers = ["Full Name", "Address", "City", "State", "Email"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))


+-----------------------+------------------------------------------+---------------------+---------+-------------------------------+
| Full Name             | Address                                  | City                | State   | Email                         |
+=======================+==========================================+=====================+=========+===============================+
| Aaron Mitchell        | 696 Osborne Street                       | Winnipeg            | MB      | aaronmitchell@yahoo.ca        |
+-----------------------+------------------------------------------+---------------------+---------+-------------------------------+
| Alexandre Rocha       | Av. Paulista, 2022                       | São Paulo           | SP      | alero@uol.com.br              |
+-----------------------+------------------------------------------+---------------------+---------+-------------------------------+
| Astrid Gruber         | Rotenturmstraße 4, 1010 Innere Stadt     | 

### Q2. Get a list of customers, sales reps, and total transaction amounts for each customer between 2011 and 2012.

In [ ]:
# Challenge 2

"""
CREATED BY: NAMAN PANDEY
DATE: 07/02/2024
QUESTION 2: Get a list of customers, sales reps, and total transaction amounts for each customer between 2011 and 2012.
"""

from pymongo import MongoClient
from datetime import datetime

# Connect to the MongoDB client (replace with your connection URI if not local)
client = MongoClient("mongodb://localhost:27017/")

# Select the database and collection
db = client["WSDA_Music"]
invoice_collection = db["Invoice"]

# Define the date range in the string format
start_date = '2011-01-01 00:00:00'
end_date = '2012-12-31 23:59:59'

# Define the date range
#start_date = datetime(2011, 1, 1)
#end_date = datetime(2012, 12, 31)

# Aggregation pipeline
pipeline = [
    {
        "$match": {
            "InvoiceDate": {"$gte": start_date, "$lte": end_date}
        }
    },
    {
        "$lookup": {
            "from": "Customer",
            "localField": "CustomerId",
            "foreignField": "CustomerId",
            "as": "customer"
        }
    },
    {
        "$unwind": "$customer"
    },
    {
        "$lookup": {
            "from": "Employee",
            "localField": "customer.SupportRepId",
            "foreignField": "EmployeeId",
            "as": "employee"
        }
    },
    {
        "$unwind": "$employee"
    },
    {
        "$group": {
            "_id": {
                "City": "$customer.City",
                "State": "$customer.State",
                "FirstName": "$customer.FirstName",
                "LastName": "$customer.LastName",
                "SalesRepFirstName": "$employee.FirstName",
                "SalesRepLastName": "$employee.LastName"
            },
            "Email": {"$first": "$customer.Email"},
            "Address": {"$first": "$customer.Address"},
            "PurchasesMade": {"$sum": "$Total"}
        }
    },
    {
        "$project": {
            "Name": {"$concat": ["$_id.FirstName", " ", "$_id.LastName"]},
            "Email": 1,
            "Address": {"$concat": ["$Address", ", ", "$_id.City", ", ", "$_id.State"]},
            "SalesRep": {"$concat": ["$_id.SalesRepFirstName", " ", "$_id.SalesRepLastName"]},
            "PurchasesMade": 1,
            "_id": 0
        }
    },
    {
        "$sort": {"Name": 1}
    }
]

# Execute the aggregation pipeline
results = list(db.Invoice.aggregate(pipeline))

# Print the results
for result in results:
    name = f"{result['Name']}"
    email = result['Email']
    address = result['Address']
    sales_rep = result['SalesRep']
    purchases_made = result['PurchasesMade']

    print(f"Name: {name}")
    print(f"Email: {email}")
    print(f"Address: {address}")
    print(f"Sales Rep: {sales_rep}")
    print(f"Purchases Made: {purchases_made}")
    print()

#### Showing the results in tabular form

In [10]:
from tabulate import tabulate

# Execute the aggregation pipeline
results = list(db.Invoice.aggregate(pipeline))

# Prepare data for tabulate
table_data = []
for result in results:
    name = f"{result['Name']}"
    email = result['Email']
    address = result['Address']
    sales_rep = result['SalesRep']
    purchases_made = result['PurchasesMade']

    table_data.append([name, email, address, sales_rep, purchases_made])

# Print results in tabular format
headers = ["Name", "Email", "Address", "Sales Rep", "Purchases Made"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))


+-----------------------+-------------------------------+----------------------------------------------------------+---------------+------------------+
| Name                  | Email                         | Address                                                  | Sales Rep     |   Purchases Made |
+=======================+===============================+==========================================================+===============+==================+
| Aaron Mitchell        | aaronmitchell@yahoo.ca        | 696 Osborne Street, Winnipeg, MB                         | Margaret Park |            11.88 |
+-----------------------+-------------------------------+----------------------------------------------------------+---------------+------------------+
| Alexandre Rocha       | alero@uol.com.br              | Av. Paulista, 2022, São Paulo, SP                        | Steve Johnson |            11.88 |
+-----------------------+-------------------------------+-------------------------------

### Q3: How many transactions are above the average transaction amount during the same time period?

In [ ]:
# Define the date range in the string format
start_date = '2011-01-01 00:00:00'
end_date = '2012-12-31 23:59:59'

# Calculate the average total for the specified date range
average_total_pipeline = [
    {
        "$match": {
            # Filter documents within the specified date range
            "InvoiceDate": {"$gte": start_date, "$lte": end_date}
        }
    },
    {
        "$group": {
            "_id": None,
            # Calculate the average total
            "average_total": {"$avg": "$Total"}
        }
    }
]

# Execute the aggregation pipeline
average_total_result = list(db.Invoice.aggregate(average_total_pipeline))
# Extract the average total from the result
average_total = average_total_result[0]['average_total'] if average_total_result else 0

print(f"Average Total is {average_total}")

# Aggregation pipeline to count transactions above average
pipeline = [
    {
        "$match": {
            # Filter documents where the total is greater than the average total
            "Total": {"$gt": average_total},
             # Filter documents within the specified date range
            "InvoiceDate": {"$gte": start_date, "$lte": end_date}
        }
    },
    {
        # Count the number of documents that match the filter
        "$count": "Transactions above Average"
    }
]

# Execute the aggregation pipeline
transactions_above_average = list(db.Invoice.aggregate(pipeline))

print(f"{transactions_above_average[0]['Transactions above Average']} transactions are above average.")

### Q4: What is the average transaction amount for each year that WSDA Music has been in business?

In [15]:
# Aggregation pipeline to calculate average transaction amount by year
pipeline = [
    {
        "$group": {
            "_id": {
                "$substr": ["$InvoiceDate", 0, 4]  # Extract the first 4 characters (the year) from the InvoiceDate field
            },
            "Average Transaction Amount": {"$avg": "$Total"}  # Calculate average total for each year
        }
    },
    {
        "$project": {
            "_id": 0,  # Exclude _id field from the result
            "Year": "$_id",  # Rename _id to Year
            "Average Transaction Amount": {"$round": ["$Average Transaction Amount", 2]}  # Round average total to two decimal places
        }
    },
    {
        "$sort": {"Year": 1}  # Sort results by year in ascending order
    }
]

# Execute the aggregation pipeline
average_transaction_amount_by_year = list(db.Invoice.aggregate(pipeline))

# Print the results
for result in average_transaction_amount_by_year:
    print(result)


{'Year': '2009', 'Average Transaction Amount': 5.42}
{'Year': '2010', 'Average Transaction Amount': 5.8}
{'Year': '2011', 'Average Transaction Amount': 17.51}
{'Year': '2012', 'Average Transaction Amount': 5.75}
{'Year': '2013', 'Average Transaction Amount': 5.63}


#### Show results in tabular form

In [18]:
from tabulate import tabulate

# Execute the aggregation pipeline
average_transaction_amount_by_year = list(db.Invoice.aggregate(pipeline))

# Prepare data for tabulate
table_data = []
for result in average_transaction_amount_by_year:
    row = [result['Year'], result['Average Transaction Amount']]
    table_data.append(row)

# Print results in tabular format
headers = ["Year", "Average Transaction Amount"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+--------+------------------------------+
|   Year |   Average Transaction Amount |
+========+==============================+
|   2009 |                         5.42 |
+--------+------------------------------+
|   2010 |                         5.8  |
+--------+------------------------------+
|   2011 |                        17.51 |
+--------+------------------------------+
|   2012 |                         5.75 |
+--------+------------------------------+
|   2013 |                         5.63 |
+--------+------------------------------+


## Challenge 3

### Q1: Get a list of employees who exceeded the average transaction amount from sales they generated during 2011 and 2012.

In [19]:
# Define the date range in the string format
start_date = '2011-01-01 00:00:00'
end_date = '2012-12-31 23:59:59'

# Pipeline to get the average sales during 2011 and 2012
average_sales_pipeline = [
    {
        "$match" : {
            "InvoiceDate": {
                "$gte": start_date,
                "$lte": end_date
            }
        }
    },
    {
        "$group": {
            "_id": None,
            "average_total": {
                "$avg": "$Total"
            }
        }
    }
]

# Execute the average sales pipeline
average_sales_result = list(db.Invoice.aggregate(average_sales_pipeline))
average_sales = average_sales_result[0]['average_total']

print(f"Average transaction amount from sales during 2011 and 2012 is {average_sales}")

# Pipeline to get employees who exceeded this average sales amount
exceeding_employees_pipeline = [
    {
        "$match": {
            "Total": {
                "$gt": average_sales,
            },
            "InvoiceDate": {
                "$gte": start_date,
                "$lte": end_date
            }
        }
    },
    {
        "$lookup": {
            "from": "Customer",
            "localField": "CustomerId",
            "foreignField": "CustomerId",
            "as": "customer"
        }
    },
    {
        "$unwind": "$customer"
    },
    {
        "$lookup": {
            "from": "Employee",
            "localField": "customer.SupportRepId",
            "foreignField": "EmployeeId",
            "as": "employee"
        }
    },
    {
        "$unwind": "$employee"
    },
    {
        "$group": {
            "_id": "$employee.EmployeeId",
            "firstname": {
                "$first": "$employee.FirstName"
                },
            "lastname": {
                "$first": "$employee.LastName"
                },
            "Employee ID": {
                "$first": "$employee.EmployeeId"
                },
            "salesMade": {
                "$sum": "$Total"
            }
        }
    },
    {
        "$sort": {"First Name": 1}
    }
]

# Execute the average sales pipeline
exceeding_employees_result = list(db.Invoice.aggregate(exceeding_employees_pipeline))

# Print the results
for employee in exceeding_employees_result:
    print(employee)

Average transaction amount from sales during 2011 and 2012 is 11.664491017964073
{'_id': 3, 'firstname': 'Jane', 'lastname': 'Peacock', 'Employee ID': 3, 'salesMade': 1137.65}
{'_id': 4, 'firstname': 'Margaret', 'lastname': 'Park', 'Employee ID': 4, 'salesMade': 139.74}
{'_id': 5, 'firstname': 'Steve', 'lastname': 'Johnson', 'Employee ID': 5, 'salesMade': 100.1}


#### Displaying the results in a tabular manner

In [20]:
from tabulate import tabulate

table_data = []
for employee in exceeding_employees_result:
    row = [employee["Employee ID"],
           employee["firstname"],
           employee["lastname"],
           employee["salesMade"]]
    table_data.append(row)

# Print results in tabular format
headers = ["Employee ID", "First Name", "Last Name", "Total Sales Made"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+---------------+--------------+-------------+--------------------+
|   Employee ID | First Name   | Last Name   |   Total Sales Made |
+===============+==============+=============+====================+
|             3 | Jane         | Peacock     |            1137.65 |
+---------------+--------------+-------------+--------------------+
|             4 | Margaret     | Park        |             139.74 |
+---------------+--------------+-------------+--------------------+
|             5 | Steve        | Johnson     |             100.1  |
+---------------+--------------+-------------+--------------------+


### Q2: Create a Commission Payout column that displays each employee’s commission based on 15% of the sales transaction amount.

In [29]:
# Define the date range in the string format
start_date = '2011-01-01 00:00:00'
end_date = '2012-12-31 23:59:59'

# Pipeline to print the commision payout at 15% of total sales
commision_payout_pipeline = [
    {
        "$match": {
            "InvoiceDate": {
                "$gte": start_date,
                "$lte": end_date
            }
        }
    },
    {
        "$lookup": {
            "from": "Customer",
            "localField": "CustomerId",
            "foreignField": "CustomerId",
            "as": "customer"
        }
    },
    {
        "$unwind": "$customer"
    },
    {
        "$lookup": {
            "from": "Employee",
            "localField": "customer.SupportRepId",
            "foreignField": "EmployeeId",
            "as": "employee"
        }
    },
    {
        "$unwind": "$employee"
    },
    {
        "$group": {
            "_id": "$employee.EmployeeId",
            "firstname": {
                "$first": "$employee.FirstName"
                },
            "lastname": {
                "$first": "$employee.LastName"
                },
            "Employee ID": {
                "$first": "$employee.EmployeeId"
                },
            "salesMade": {
                "$sum": "$Total"
            }
        }
    },
    {
        "$addFields": { # Calculate commission payout (15% of total sales) and round to 2 decimal places
            "commisionPayout": {
                "$round": [
                    {"$multiply": ["$salesMade", 0.15]},
                    2]
            }
        }
    },
    {
        "$sort": {"First Name": 1}
    }
]

# Execute the average sales pipeline
commision_payout_results = list(db.Invoice.aggregate(commision_payout_pipeline))

# Print the results
for employee in commision_payout_results:
    print(employee)

{'_id': 3, 'firstname': 'Jane', 'lastname': 'Peacock', 'Employee ID': 3, 'salesMade': 1331.8, 'commisionPayout': 199.77}
{'_id': 5, 'firstname': 'Steve', 'lastname': 'Johnson', 'Employee ID': 5, 'salesMade': 293.2, 'commisionPayout': 43.98}
{'_id': 4, 'firstname': 'Margaret', 'lastname': 'Park', 'Employee ID': 4, 'salesMade': 322.96999999999997, 'commisionPayout': 48.45}


#### Displaying the results in a tabular manner

In [30]:
from tabulate import tabulate

table_data = []
for employee in commision_payout_results:
    row = [employee["Employee ID"],
           employee["firstname"],
           employee["lastname"],
           employee["salesMade"],
           employee["commisionPayout"]]
    table_data.append(row)

# Print results in tabular format
headers = ["Employee ID", "First Name", "Last Name", "Total Sales Made", "Commision Payout"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+---------------+--------------+-------------+--------------------+--------------------+
|   Employee ID | First Name   | Last Name   |   Total Sales Made |   Commision Payout |
+===============+==============+=============+====================+====================+
|             3 | Jane         | Peacock     |            1331.8  |             199.77 |
+---------------+--------------+-------------+--------------------+--------------------+
|             5 | Steve        | Johnson     |             293.2  |              43.98 |
+---------------+--------------+-------------+--------------------+--------------------+
|             4 | Margaret     | Park        |             322.97 |              48.45 |
+---------------+--------------+-------------+--------------------+--------------------+


### Q3: Which employee made the highest commission?

In [32]:
# Define the date range in the string format
start_date = '2011-01-01 00:00:00'
end_date = '2012-12-31 23:59:59'

# Pipeline to print the employee who made the highest commision payout
commision_payout_pipeline = [
    {
        "$match": {
            "InvoiceDate": {
                "$gte": start_date,
                "$lte": end_date
            }
        }
    },
    {
        "$lookup": {
            "from": "Customer",
            "localField": "CustomerId",
            "foreignField": "CustomerId",
            "as": "customer"
        }
    },
    {
        "$unwind": "$customer"
    },
    {
        "$lookup": {
            "from": "Employee",
            "localField": "customer.SupportRepId",
            "foreignField": "EmployeeId",
            "as": "employee"
        }
    },
    {
        "$unwind": "$employee"
    },
    {
        "$group": {
            "_id": "$employee.EmployeeId",
            "firstname": {
                "$first": "$employee.FirstName"
                },
            "lastname": {
                "$first": "$employee.LastName"
                },
            "Employee ID": {
                "$first": "$employee.EmployeeId"
                },
            "salesMade": {
                "$sum": "$Total"
            }
        }
    },
    {
        "$addFields": { # Calculate commission payout (15% of total sales) and round to 2 decimal places
            "commisionPayout": {
                "$round": [
                    {"$multiply": ["$salesMade", 0.15]},
                    2]
            }
        }
    },
    {
        # Sorting the commision payout in descending order
        "$sort": {"commisionPayout": -1}
    },
    {
        # limiting the results returned to only the top
        "$limit": 1
    }
]

# Execute the average sales pipeline
commision_payout_results = list(db.Invoice.aggregate(commision_payout_pipeline))

# Print the results
for employee in commision_payout_results:
    print(employee)

{'_id': 3, 'firstname': 'Jane', 'lastname': 'Peacock', 'Employee ID': 3, 'salesMade': 1331.8, 'commisionPayout': 199.77}


### Q4: List the customers that the employee identified in the last question.

In [38]:
# Write the query for filtering customers whose support rep is Jane (SupportRepId is 3)
query = {"SupportRepId": 3}

# Executing that query
results = db.Customer.find(query)

# Printing results
for customer in results:
    print(customer)


{'_id': ObjectId('65b67eab36403a5660e6ca0e'), 'CustomerId': 1, 'FirstName': 'Luís', 'LastName': 'Gonçalves', 'Company': 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Address': 'Av. Brigadeiro Faria Lima, 2170', 'City': 'São José dos Campos', 'State': 'SP', 'Country': 'Brazil', 'PostalCode': '12227-000', 'Phone': '+55 (12) 3923-5555', 'Fax': '+55 (12) 3923-5566', 'Email': 'luisg@embraer.com.br', 'SupportRepId': 3}
{'_id': ObjectId('65b67eab36403a5660e6ca10'), 'CustomerId': 3, 'FirstName': 'François', 'LastName': 'Tremblay', 'Address': '1498 rue Bélanger', 'City': 'Montréal', 'State': 'QC', 'Country': 'Canada', 'PostalCode': 'H2G 1A7', 'Phone': '+1 (514) 721-4711', 'Email': 'ftremblay@gmail.com', 'SupportRepId': 3}
{'_id': ObjectId('65b67eab36403a5660e6ca19'), 'CustomerId': 12, 'FirstName': 'Roberto', 'LastName': 'Almeida', 'Company': 'Riotur', 'Address': 'Praça Pio X, 119', 'City': 'Rio de Janeiro', 'State': 'RJ', 'Country': 'Brazil', 'PostalCode': '20040-020', 'Phone': '+55 (21)

#### Displaying in Tabular Form

In [54]:
# Write the query for filtering customers whose support rep is Jane (SupportRepId is 3)
query = {"SupportRepId": 3}

# Executing that query
results = db.Customer.find(query)

table_data = []
for customer in results:
    # Access fields using .get() method to handle missing fields gracefully
    row = [
        customer.get("CustomerId", ""),
        customer.get("FirstName", "") + " " + customer.get("LastName", ""),
        (customer.get("Address", "") + " " + customer.get("City", "") + " " + customer.get("State", "") + " " + customer.get("Country", "")).strip(),
        customer.get("Email", ""),
        customer.get("Phone", "")
    ]
    table_data.append(row)

headers = ["Customer ID", "Name", "Address", "Email", "Phone"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))


+---------------+-------------------+---------------------------------------------------------------+-------------------------------+--------------------+
|   Customer ID | Name              | Address                                                       | Email                         | Phone              |
+===============+===================+===============================================================+===============================+====================+
|             1 | Luís Gonçalves    | Av. Brigadeiro Faria Lima, 2170 São José dos Campos SP Brazil | luisg@embraer.com.br          | +55 (12) 3923-5555 |
+---------------+-------------------+---------------------------------------------------------------+-------------------------------+--------------------+
|             3 | François Tremblay | 1498 rue Bélanger Montréal QC Canada                          | ftremblay@gmail.com           | +1 (514) 721-4711  |
+---------------+-------------------+---------------------------------

### Q5: Which customer made the highest purchase?

In [63]:
# pipeline to retrieve the customer who made the highest purchase
purchases_pipeline = [
    {
        "$lookup": {
            "from": "Customer",
            "localField": "CustomerId",
            "foreignField": "CustomerId",
            "as": "customer"
        }
    },
    {
        "$unwind": "$customer"
    },
    {
        "$group": {
            "_id": "$customer.CustomerId",
            "firstname": {
                "$first": "$customer.FirstName"
                },
            "lastname": {
                "$first": "$customer.LastName"
                },
            "purchasesMade": {
                "$sum": "$Total"
            }
        }
    },
    {
        "$sort": {"purchasesMade": -1}
    },
    {
        "$limit": 1
    }
]

# Execute the pipeline
purchases_results = list(db.Invoice.aggregate(purchases_pipeline))

# Print the result
print(purchases_results)

[{'_id': 60, 'firstname': 'John', 'lastname': 'Doeein', 'purchasesMade': 1000.86}]


#### Based on the above, we can see John Doeein has no details and has made extremely high purchases which could point out that it is a false record, created by Jane Peacock. Hence the financial discrepancy could be attributed to Jane Peacock.